<p class="float-right">![logo](https://raw.githubusercontent.com/mmattamala/LogosFCFM/master/Ingenier%C3%ADa%20Matem%C3%A1tica/fcfm_dim_png.png)</p>

# Tarea 1


MA5203 - Aprendizaje de Máquinas Probabilístico

Profesor: Felipe Tobar

Autor: Martín Cornejo